# A simple "Hello World" tutorial for Distributed Systems' REST APIs 

## What we will do
* Run a Hello world example on http://localhost:5000
* Create a list of todos
* Allow to tinker with those todos


## Step 0: Prerequisites

First of all, we need to import the require libraries. We will be using Flask and Flask RESTFUL.
* Flask is a microframework for Python based on Werkzeug and Jinja 2. It allows creating a web application in a few lines of code. A microframework means its lean and mean (see next week)
* Flask RESTFUL enables you to turn your Flask web applications into REST APIs


So lets start importing Flask (we call it Flask) and the flask_restful api objects

In [ ]:
from flask import Flask
from flask_restful import reqparse, abort, Api, Resource

We will make an app (to create the web application) and an api to refer to the REST API. In the future we will mainly add resources to this api 

In [ ]:
app = Flask(__name__)
api = Api(app)

### Only relevant for this demo (simply ignore)

Since we already started the server, we need to shut it down again if we want to add endpoints. Specifically for this demo this is important, but you should not care about this yourself in your code. 


In [ ]:
from flask import request
def shutdown_server():
    func = request.environ.get('werkzeug.server.shutdown')
    if func is None:
        raise RuntimeError('Not running with the Werkzeug Server')
    func()

@app.route('/shutdown', methods=['GET'])
def shutdown():
    shutdown_server()
    return 'Server shutting down...'

## Step 1: Hello world

Every new beginning starts with a hello world.

For this we will make a 'HelloWorld' class. The method get will return a JSON object with key/value hello and world. 

In [ ]:
class HelloWorld(Resource):
    def get(self):
        return {'hello': 'world'}

The only thing we need to do now is to add this class HelloWorld to our API as a resource. The second argument defines the URL at where this will run. In this case, this is the root.

In [ ]:
api.add_resource(HelloWorld, '/')

Time to spin up our API. 

In [ ]:
if __name__ == '__main__':
    app.run()

Because of the code I inserted above, I can now simply shutdown the server by going to http://localhost:5000/shutdown

## Step 2: Todos
That was fun but way too easy. Lets make things more complex. 

We will create our own simple TODO manager, which supports the following things
* Get a list of todos
* Get a particular todo
* Add a todo
* Delete a todo


Lets start with creating a bunch of sample todos

In [ ]:
TODOS = {
    'todo1': {'task': 'Teach Distributed Systems'},
    'todo2': {'task': 'Think of a good joke to tell during Distributed Systems'},
    'todo3': {'task': 'It\'s friday! Order fries and a Bicky Burger.'},
}

And a dummy proof method.

In [ ]:
def abort_if_todo_doesnt_exist(todo_id):
    if todo_id not in TODOS:
        abort(404, message="Todo {} doesn't exist".format(todo_id))


If we want to do something interesting with our REST API we need to be able to parse arguments we get from our clients. While Flask provides easy access to request data (i.e. querystring or POST form encoded data), it’s still a pain to validate form data. Flask-RESTful has built-in support for request data validation using the library reqparse (which we already imported above).



In [ ]:

parser = reqparse.RequestParser()
parser.add_argument('task')


Here, task is going to be an argument which we can pass around to our REST API to either retrieve a todo or delete it. 

The next thing we want to do is support the todos endpoint. This endpoint will allow us to 
* GET a list of our pending todos (when we call an HTTP GET)
* ADD a specific todo (when we call HTTP POST). In this case, we will add the todo ID ourselves.]

Similar to our HelloWorld example, the TodoList class will define get and post methods to do all of that. 

The get method simply returns our object TODOS

In [ ]:
# TodoList
# shows a list of all todos, and lets you POST to add new tasks
class TodoList(Resource):
    def get(self):
        return TODOS

    

The post method will 
* Parse the arguments
* Find the next todo ID
* Add a todo to the array (based on the task parameter)

In [ ]:
def post(self):
        args = parser.parse_args()
        todo_id = int(max(TODOS.keys()).lstrip('todo')) + 1
        todo_id = 'todo%i' % todo_id
        TODOS[todo_id] = {'task': args['task']}
        return TODOS[todo_id], 201


The second endpoint is the ability to
* Retrieve a specific todo (HTTP GET)
* Delete a todo (HTTP DELETE)
* Add a todo (HTTP PUT)

Same story here: just add the methods. Note that in this case we use the todo_id as an argument in the URL. The actual task name is still encoded as a parameter

In [ ]:
# Todo
# shows a single todo item and lets you delete a todo item
class Todo(Resource):
    def get(self, todo_id):
        abort_if_todo_doesnt_exist(todo_id)
        return TODOS[todo_id]

    def delete(self, todo_id):
        abort_if_todo_doesnt_exist(todo_id)
        del TODOS[todo_id]
        return '', 204

    def put(self, todo_id):
        args = parser.parse_args()
        task = {'task': args['task']}
        TODOS[todo_id] = task
        return task, 201





The last step is then to add our resources. Note that we define the todo_id here

In [ ]:
##
## Actually setup the Api resource routing here
##
api.add_resource(TodoList, '/todos')
api.add_resource(Todo, '/todos/<todo_id>')

And just run the server again. 

In [ ]:
if __name__ == '__main__':
    app.run()